# Exploratory Data Analysis & Data Preprocessing
---

Nhiệm vụ của phần này là làm sạch và khám phá dữ liệu

## IMPORT

In [1]:
# Import các thư viện cần thiết
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder

## ĐỌC DỮ LIỆU

In [2]:
# Google Colab
# url = "https://raw.githubusercontent.com/uyen312/Project_PTDLTM/main/Data/dataset.csv"
# df = pd.read_csv(url)

# VSCode
df = pd.read_csv("dataset.csv")

In [3]:
df.head(5)

,id,sale_date,sale_price,sale_nbr,sale_warning,join_status,join_year,latitude,longitude,area,...,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_lakesamm,view_otherwater,view_other,submarket
0,0,2014-11-15,236000,2.0,,nochg,2025,47.2917,-122.3658,53,...,0,0,0,0,0,0,0,0,0,I
1,1,1999-01-15,313300,NaN,26,nochg,2025,47.6531,-122.1996,74,...,0,0,0,0,0,1,0,0,0,Q
2,2,2006-08-15,341000,1.0,,nochg,2025,47.4733,-122.1901,30,...,0,0,0,0,0,0,0,0,0,K
3,3,1999-12-15,267000,1.0,,nochg,2025,47.4739,-122.3295,96,...,0,0,0,0,0,0,0,0,0,G
4,4,2018-07-15,1650000,2.0,,miss99,2025,47.7516,-122.1222,36,...,0,0,0,0,0,0,0,0,0,P


## LÀM SẠCH VÀ PHÂN TÍCH DỮ LIỆU

### Dữ liệu có bao nhiêu dòng, cột

In [4]:
df.shape

(200000, 47)

### Thông tin về các dòng dữ liệu

#### Ý nghĩa từng dòng dữ liệu

- Mỗi dòng là đại diện cho ....

- Không có dòng nào có ý nghĩa khác với các dòng còn lại.

#### Kiểm tra các dòng dữ liệu trùng lặp

In [5]:
duplicate_rows = df.duplicated().sum()
duplicate_rows

0

Dữ liệu không có các dòng trùng lặp

### Thông tin về các cột dữ liệu
Dữ liệu có các cột sau

In [6]:
df.columns

Index(['id', 'sale_date', 'sale_price', 'sale_nbr', 'sale_warning',
       'join_status', 'join_year', 'latitude', 'longitude', 'area', 'city',
       'zoning', 'subdivision', 'present_use', 'land_val', 'imp_val',
       'year_built', 'year_reno', 'sqft_lot', 'sqft', 'sqft_1', 'sqft_fbsmt',
       'grade', 'fbsmt_grade', 'condition', 'stories', 'beds', 'bath_full',
       'bath_3qtr', 'bath_half', 'garb_sqft', 'gara_sqft', 'wfnt', 'golf',
       'greenbelt', 'noise_traffic', 'view_rainier', 'view_olympics',
       'view_cascades', 'view_territorial', 'view_skyline', 'view_sound',
       'view_lakewash', 'view_lakesamm', 'view_otherwater', 'view_other',
       'submarket'],
      dtype='object')

#### Ý nghĩa của các cột dữ liệu

Mỗi cột có ý nghĩa sau: 
|Tên cột|Ý nghĩa|
|-------|-------|
| id| |

#### Kiểm tra kiểu dữ liệu của mỗi cột
Kiểu dữ liệu hiện tại của các cột

In [7]:
df.dtypes

id                    int64
sale_date            object
sale_price            int64
sale_nbr            float64
sale_warning         object
join_status          object
join_year             int64
latitude            float64
longitude           float64
area                  int64
city                 object
zoning               object
subdivision          object
present_use           int64
land_val              int64
imp_val               int64
year_built            int64
year_reno             int64
sqft_lot              int64
sqft                  int64
sqft_1                int64
sqft_fbsmt            int64
grade                 int64
fbsmt_grade           int64
condition             int64
stories             float64
beds                  int64
bath_full             int64
bath_3qtr             int64
bath_half             int64
garb_sqft             int64
gara_sqft             int64
wfnt                  int64
golf                  int64
greenbelt             int64
noise_traffic       

Cột có dữ liệu chưa phù hợp là `sale_date`, ta sẽ chuyển về kiểu dữ liệu phù hợp là datetime

In [8]:
df['sale_date'] = pd.to_datetime(df['sale_date'], errors='coerce')

#### Sự phân bố dữ liệu của các cột có kiểu định lượng
Các cột có dữ liệu định lượng

In [9]:
numerical_columns = df.select_dtypes(include=['number']).columns.tolist()
print('Các cột có kiểu dữ liệu định lượng: ')
for col_name in numerical_columns:
    print(col_name)

Các cột có kiểu dữ liệu định lượng: 
id
sale_price
sale_nbr
join_year
latitude
longitude
area
present_use
land_val
imp_val
year_built
year_reno
sqft_lot
sqft
sqft_1
sqft_fbsmt
grade
fbsmt_grade
condition
stories
beds
bath_full
bath_3qtr
bath_half
garb_sqft
gara_sqft
wfnt
golf
greenbelt
noise_traffic
view_rainier
view_olympics
view_cascades
view_territorial
view_skyline
view_sound
view_lakewash
view_lakesamm
view_otherwater
view_other


##### Tính toán các giá trị thống kê và xử lý giá trị thiếu

Với mỗi cột có kiểu dữ liệu định lượng, thực hiện tính toán các giá trị thống kê cơ bản sau:
- Phần trăm (từ 0 đến 100) của giá trị thiếu
- Giá trị nhỏ nhất (Min)
- Tứ phân vị dưới (Lower quartile)
- Trung vị (Median)
- Tứ phân vị trên (Upper quartile)
- Giá trị lớn nhất (Max)

Các giá trị được làm tròn đến chữ số thập phân thứ nhất.

In [10]:
# Chọn ra những cột có kiểu định lượng
numeric_cols = df.select_dtypes(include=['number']).columns

# Tính toán các giá trị thống kê
def missing_ratio(series):
    return series.isnull().mean() * 100

def lower_quartile(series):
    return series.quantile(0.25)

def median(series):
    return series.median()

def upper_quartile(series):
    return series.quantile(0.75)

min_vals = df[numeric_cols].min()
max_vals = df[numeric_cols].max()
lower_quartiles = df[numeric_cols].apply(lower_quartile)
medians = df[numeric_cols].median()
upper_quartiles = df[numeric_cols].apply(upper_quartile)

# Tạo một DataFrame chứa các giá trị thống kê của những cột định lượng
num_col_info_df = pd.DataFrame({
    "Tỷ lệ giá trị thiếu": df[numeric_cols].apply(missing_ratio).round(1),
    "Giá trị nhỏ nhất": min_vals.round(1),
    "Tứ phân vị dưới": lower_quartiles.round(1),
    "Trung vị": medians.round(1),
    "Tứ phân vị trên": upper_quartiles.round(1),
    "Giá trị lớn nhất": max_vals.round(1)
}).T

# In ra
num_col_info_df = num_col_info_df.T  # Đảo chiều
num_col_info_df

,Tỷ lệ giá trị thiếu,Giá trị nhỏ nhất,Tứ phân vị dưới,Trung vị,Tứ phân vị trên,Giá trị lớn nhất
id,0.0,0.0,49999.8,99999.5,149999.2,199999.0
sale_price,0.0,50293.0,305000.0,459950.0,724950.0,2999950.0
sale_nbr,21.1,1.0,1.0,2.0,3.0,11.0
join_year,0.0,1999.0,2025.0,2025.0,2025.0,2025.0
latitude,0.0,47.2,47.4,47.6,47.7,47.8
longitude,0.0,-122.5,-122.3,-122.2,-122.1,-121.2
area,0.0,1.0,26.0,48.0,71.0,100.0
present_use,0.0,2.0,2.0,2.0,2.0,29.0
land_val,0.0,0.0,231000.0,377000.0,594000.0,13864000.0
imp_val,0.0,0.0,280000.0,409000.0,599000.0,10067000.0


**Nhận xét**: Một cột chứa giá trị thiếu là `sale_nbr` với tỷ lệ lên đến 21%. 

Vì không có thông tin về số lần bất động sản đã được bán trước đó, ta tạm giả định rằng những dòng thiếu dữ liệu ở cột sale_nbr là các trường hợp chưa từng được bán (tức là nhà mới hoàn toàn). Do đó, ta sẽ:

- Điền các giá trị thiếu trong sale_nbr bằng 0.

- Đồng thời tạo thêm một cột mới (sale_nbr_missing) để đánh dấu các dòng mà dữ liệu ban đầu bị thiếu, nhằm giữ lại thông tin này cho các bước phân tích và mô hình hóa sau.

In [11]:
df['sale_nbr_missing'] = df['sale_nbr'].isnull().astype(int)

df['sale_nbr'] = df['sale_nbr'].fillna(0)

##### Sự phân bố dữ liệu

In [12]:
# Your code here

##### Xử lý dữ liệu lỗi hoặc bất thường

In [13]:
# Hàm đếm outlier dựa trên quy tắc Tukey
def count_outliers(data, column):
    Q1 = data[column].quantile(0.25)  # Phân vị thứ 25
    Q3 = data[column].quantile(0.75)  # Phân vị thứ 75
    IQR = Q3 - Q1                    # Khoảng tứ phân vị (IQR)
    lower_bound = Q1 - 1.5 * IQR     # Ngưỡng dưới
    upper_bound = Q3 + 1.5 * IQR     # Ngưỡng trên
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]

    # Kiểm tra nếu ngưỡng dưới âm và điều chỉnh
    if lower_bound < 0:
        lower_bound = 0  # Đặt lại ngưỡng dưới bằng 0 vì nếu mang giá trị âm sẽ không hợp lý với dữ liệu
        
    return len(outliers)

def count_outliers_for_all_columns(data, numerical_columns):
    outliers_info = {}
    for column in numerical_columns:
        outlier_count = count_outliers(data, column)

        # Lưu kết quả vào info
        outliers_info[column] = {
            'Số giá trị ngoại lai': outlier_count,
            'Tỷ lệ giá trị ngoại lai': outlier_count / data.shape[0]
        }

    return outliers_info

# In kết quả
outliers_data = pd.DataFrame(count_outliers_for_all_columns(df, numerical_columns))
outliers_data = outliers_data.T  # Đảo chiều 
outliers_data 

,Số giá trị ngoại lai,Tỷ lệ giá trị ngoại lai
id,0.0,0.000000
sale_price,11736.0,0.058680
sale_nbr,18182.0,0.090910
join_year,12280.0,0.061400
latitude,0.0,0.000000
longitude,2489.0,0.012445
area,0.0,0.000000
present_use,16847.0,0.084235
land_val,7486.0,0.037430
imp_val,13931.0,0.069655


**Nhận xét**:


#### Sự phân bố dữ liệu của các cột có kiểu định tính
Các cột có kiểu dữ liệu định tính

In [14]:
categorical_columns = df.select_dtypes(exclude=['number']).columns.tolist()
print('Các cột có kiểu dữ liệu định tính: ')
for col_name in categorical_columns:
    print(col_name)

Các cột có kiểu dữ liệu định tính: 
sale_date
sale_warning
join_status
city
zoning
subdivision
submarket


#### Tỉ lệ phần trăm các giá trị bị thiếu trong mỗi cột

Với mỗi cột có kiểu dữ liệu định tính, thực hiện tính toán phần trăm (từ 0 đến 100) của giá trị thiếu

Các giá trị được làm tròn đến chữ số thập phân thứ nhất.

In [15]:
# Chọn ra các cột định tính
categorical_cols = df.select_dtypes(exclude=['number']).columns

# Hàm tính tỷ lệ giá trị thiếu
def missing_ratio(series):
    return series.isnull().mean() * 100

# Tạo DataFrame thống kê
categorical_info_df = pd.DataFrame({
    "Tỷ lệ giá trị thiếu (%)": df[categorical_cols].apply(missing_ratio).round(1),
}).T

# In kết quả
categorical_info_df

,sale_date,sale_warning,join_status,city,zoning,subdivision,submarket
Tỷ lệ giá trị thiếu (%),0.0,0.0,0.0,0.0,0.0,8.8,0.9


**Nhận xét**: Hai cột có giá trị thiếu là `subdivision` và `submarket`

Tuy tỷ lệ thiếu không quá cao (8.8% và 0.9%), việc loại bỏ các dòng chứa giá trị thiếu có thể làm mất thông tin quan trọng ở các đặc trưng khác. Do đó, ta lựa chọn điền giá trị thiếu bằng 'unknown' là một nhãn đặc biệt đại diện cho trường hợp không có thông tin ban đầu. Từ đó ta có thể:

- Giữ lại toàn bộ dữ liệu gốc để phục vụ huấn luyện mô hình học máy.

- Giữ nguyên tín hiệu về sự thiếu thông tin, vì 'unknown' sẽ trở thành một lớp phân loại riêng, cho phép mô hình học được sự khác biệt này.

- Tránh gây sai lệch phân phối hoặc mất mát dữ liệu không cần thiết.

In [16]:
df['subdivision'] = df['subdivision'].fillna('unknown')
df['submarket'] = df['submarket'].fillna('unknown')

#### Số giá trị phân biệt của mỗi cột


Với một cột dữ liệu định tính, thực hiện tính toán:
- Số giá trị phân biệt cho từng cột
- Giá trị xuất hiện nhiều nhất (mode): Tìm giá trị phổ biến nhất trong cột (nếu có nhiều giá trị bằng nhau, chọn ngẫu nhiên một giá trị).
- Tần suất xuất hiện của giá trị phổ biến nhất: Đếm số lần xuất hiện của giá trị phổ biến nhất.

In [17]:
# Hàm tính giá trị phổ biến nhất (mode) và tần suất của nó
def mode_value(series):
    if series.mode().empty:
        return None  # Nếu không có mode
    return series.mode()[0]

def mode_frequency(series):
    if series.mode().empty:
        return 0  # Nếu không có mode
    return (series == series.mode()[0]).sum()

# Tạo DataFrame thống kê
categorical_info_df = pd.DataFrame({
    "Số lượng giá trị phân biệt": df[categorical_cols].nunique(),
    "Giá trị phổ biến nhất": df[categorical_cols].apply(mode_value),
    "Tần suất của giá trị phổ biến nhất": df[categorical_cols].apply(mode_frequency)
}).T

# In kết quả
categorical_info_df

,sale_date,sale_warning,join_status,city,zoning,subdivision,submarket
Số lượng giá trị phân biệt,313,142,8,41,500,10377,20
Giá trị phổ biến nhất,2004-06-15 00:00:00,,nochg,SEATTLE,NR3,unknown,K
Tần suất của giá trị phổ biến nhất,1221,180689,126281,59001,28800,17550,21002


#### Sự phân bố dữ liệu

In [18]:
# Your code here

### Tương quan..

In [19]:
# Your code here

## SAU CÁC BƯỚC TIỀN XỬ LÝ

Số hàng, số cột của dữ liệu:

In [20]:
df.shape

(200000, 48)

Kiểu dữ liệu của các cột

In [21]:
df.dtypes

id                           int64
sale_date           datetime64[ns]
sale_price                   int64
sale_nbr                   float64
sale_warning                object
join_status                 object
join_year                    int64
latitude                   float64
longitude                  float64
area                         int64
city                        object
zoning                      object
subdivision                 object
present_use                  int64
land_val                     int64
imp_val                      int64
year_built                   int64
year_reno                    int64
sqft_lot                     int64
sqft                         int64
sqft_1                       int64
sqft_fbsmt                   int64
grade                        int64
fbsmt_grade                  int64
condition                    int64
stories                    float64
beds                         int64
bath_full                    int64
bath_3qtr           

Thực hiện lưu dữ liệu này vào 1 tệp mới và thực hiện phân tích trên tệp này.

In [22]:
df.to_csv('Dataset_clean.csv', index=False)